In [1]:
import os
from api_requests import authenticate_email_api, get_email_content, get_rescuetime_daily
from text_processing import reMarkableParsing
import pandas as pd
import re
from dateutil.parser import parse
import timeit
import sqlite3
import json
import datetime

from datetime import date
from dateutil import relativedelta

In [2]:
os.chdir('/mnt/c/Users/colta/portfolio/codex_vitae_app/config')
    
service = authenticate_email_api()
remarkable = get_email_content(service,query="from:my@remarkable.com")

journal_entries = reMarkableParsing(remarkable).run()


In [3]:
os.chdir('/mnt/c/Users/colta/portfolio/codex_vitae_app/config')
service = authenticate_email_api()
mynetdiary = get_email_content(service,query="from:no-reply@mynetdiary.net")

In [2]:
# RescueTime code for 

#logger = logging.getLogger(__name__)

os.chdir('/mnt/c/Users/colta/portfolio/codex_vitae_app/config')


with open("rescuetime_cred.json", "r") as file:
    credentials = json.load(file)
KEY = credentials.get('rescuetime').get('KEY')

rescuetime_tuple = get_rescuetime_daily(KEY)

rescuetime_tuple



[('2021-04-16', 5.24, 4.3, 2.05),
 ('2021-04-15', 5.8, 2.87, 3.19),
 ('2021-04-14', 2.7, 4.78, 2.76),
 ('2021-04-13', 1.45, 3.74, 1.99),
 ('2021-04-12', 2.63, 4.6, 2.45),
 ('2021-04-11', 0.26, 3.24, 0.98),
 ('2021-04-09', 2.27, 2.31, 1.0),
 ('2021-04-08', 6.35, 2.69, 2.34),
 ('2021-04-07', 1.16, 3.21, 1.82),
 ('2021-04-06', 1.36, 2.74, 2.13),
 ('2021-04-05', 0.52, 1.38, 1.29),
 ('2021-04-04', 0.37, 1.89, 0.27),
 ('2021-04-03', 0.11, 0.51, 0.36),
 ('2021-04-02', 0.32, 2.26, 1.58)]

In [133]:
'''THIS IS THE .PY FILE IN THE ABOVE CELL'''
#from db_utils import db_connect

con = sqlite3.connect("/mnt/c/Users/colta/portfolio/codex_vitae_app/data/testdb.sqlite") # connect to the database
cur = con.cursor() # instantiate a cursor obj to execute SQL statements

cur.execute('SELECT SQLITE_VERSION()')
data = cur.fetchone()
print("SQLite version: %s" % data) 

#save a CREATE TABLE statement to a variable
rescuetime_sql = """
CREATE TABLE rescuetime (
    date text PRIMARY KEY,
    prod_hours float,
    dist_hours float,
    neut_hours float
    )"""

#save a CREATE TABLE statement to a variable
journal_sql = """
CREATE TABLE journal (
    date text PRIMARY KEY,
    mood float,
    entry text
    )"""


#Create the table
cur.execute(rescuetime_sql)
cur.execute(journal_sql)

con.commit()
con.close()


SQLite version: 3.34.0


In [132]:
con = sqlite3.connect("/mnt/c/Users/colta/portfolio/codex_vitae_app/data/testdb.sqlite") # connect to the database
cur = con.cursor() # instantiate a cursor obj to execute SQL statements


drop_rescuetime = """
DROP TABLE rescuetime
"""

drop_journal = """
DROP TABLE journal
"""

#Create the table
cur.execute(drop_rescuetime)
cur.execute(drop_journal)
con.commit()
con.close()

In [146]:
def insert_rescuetime(time_tuple):
    con = sqlite3.connect("/mnt/c/Users/colta/portfolio/codex_vitae_app/data/testdb.sqlite") # connect to the database
    cur = con.cursor() # instantiate a cursor obj to execute SQL statements
    sql = """
        INSERT INTO rescuetime
            (date, prod_hours, dist_hours, neut_hours)
        VALUES (?, ?, ?, ?)
        ON CONFLICT(date) DO UPDATE SET date = excluded.date
        """
    con.row_factory = sqlite3.Row
    for row in time_tuple:
        cur.execute(sql,(row[0],row[1],row[2],row[3]))
    con.commit()
    con.close()
    return cur.lastrowid


def insert_journal(journal_tuple):
    con = sqlite3.connect("/mnt/c/Users/colta/portfolio/codex_vitae_app/data/testdb.sqlite") # connect to the database
    cur = con.cursor() # instantiate a cursor obj to execute SQL statements
    sql = """
        INSERT INTO journal
            (date, mood, entry)
        VALUES (?, ?, ?)
        ON CONFLICT(date) DO UPDATE SET date = excluded.date
        """
    con.row_factory = sqlite3.Row
    for row in journal_tuple:
        cur.execute(sql,(row[0],row[1],row[2]))
    con.commit()
    con.close()
    return cur.lastrowid

In [154]:
#%%timeit
insert_rescuetime(rescuetime_tuple)

0

In [153]:
insert_journal(journal_tuple)


0

In [161]:
%%timeit
con = sqlite3.connect("/mnt/c/Users/colta/portfolio/codex_vitae_app/data/testdb.sqlite") # connect to the database
test_rescuetime = pd.read_sql_query("SELECT * from rescuetime",con)
con.close()

The slowest run took 4.06 times longer than the fastest. This could mean that an intermediate result is being cached.
15.3 ms ± 6.53 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [162]:
%%timeit

import datetime 

con = sqlite3.connect("/mnt/c/Users/colta/portfolio/codex_vitae_app/data/testdb.sqlite") # connect to the database
cur = con.cursor() # instantiate a cursor obj to execute SQL statements

cur.execute("SELECT * FROM rescuetime ORDER BY date")
results = cur.fetchall()

con.commit()
con.close()

day = [datetime.datetime.strptime(row[0],'%Y-%m-%d') for row in results]
all_hrs = [row[1] for row in results]
prd_hrs = [row[2] for row in results]
dst_hrs = [row[3] for row in results]
nt_hrs = [row[4] for row in results]


9.56 ms ± 1.38 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [173]:
con = sqlite3.connect("/mnt/c/Users/colta/portfolio/codex_vitae_app/data/testdb.sqlite") # connect to the database
cur = con.cursor() # instantiate a cursor obj to execute SQL statements

cur.execute("SELECT * FROM journal ORDER BY date")
results = cur.fetchall()

con.commit()
con.close()

day = [datetime.datetime.strptime(row[0],'%Y-%m-%d') for row in results]
mood = [row[1] for row in results]
journal = [row[2] for row in results]